In [1]:
from dotenv import load_dotenv
import os
import requests
import base64

load_dotenv()

# Configuration
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

# Define the URL of the image
url_shelf_image = os.getenv('URL_SHELF_IMAGE')
url_fanta_pineapple_image= os.getenv('URL_FANTA_PINEAPPLE_IMAGE')
url_sprite_image= os.getenv('URL_SPRITE_IMAGE')
url_dasani_image= os.getenv('URL_DASANI_IMAGE')

# Download the image
response_shelf_image = requests.get(url_shelf_image)
response_fanta_pineapple_image = requests.get(url_fanta_pineapple_image)
response_sprite_image = requests.get(url_sprite_image)
response_dasani_image = requests.get(url_dasani_image)


encoded_shelf_image = base64.b64encode(response_shelf_image.content).decode('ascii')
encoded_fanta_pineapple_image = base64.b64encode(response_fanta_pineapple_image.content).decode('ascii')
encoded_sprite_image = base64.b64encode(response_sprite_image.content).decode('ascii')
encoded_dasani_image = base64.b64encode(response_dasani_image.content).decode('ascii')



headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

In [3]:
# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are a retail Coca-cola planogram and assortment auditor.  Our products SKUs are: Coca-cola Original can 330ml, Coca-cola Zero can 330ml, Fanta Orange can 330ml, Sprite Original can 330ml, Sprite Zero can 330ml, Dasani water bottle 500ml.  Your job is to exhaustively inspect each shelf of the cooler photo and audit to determine if the cooler is compliant or not by shelf and if there are products from competitors or different products."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "TASK:  you are going to create a single audit table by each shelf with columns: Shelf Number, Required SKUs, Compliance summary with counts per product by comparing the below RULES of the PLANOGRAM with the above attached image, the image has 6 shelfs.  DO NOT hallucinate with counts, include only products with numbers on the same shelf and same SKU similarity."
        },
        {
          "type": "text",
          "text": "\n"
        },
        {
          "type": "text",
          "text": "PLANOGRAM RULES:"
        },
        {
          "type": "text",
          "text": "Shelf 1 (top):"
        },
        {
          "type": "text",
          "text": "\t- Must have 2 units of Coca-Cola Original bottle 330ml"
        },
        {
          "type": "text",
          "text": "\t- Must have 2 units of Coca-Cola Zero bottle 330ml"
        },
        {
          "type": "text",
          "text": "\t- Must have 2 units of Sprite bottle 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 2:"
        },
        {
          "type": "text",
          "text": "\t- Must have 2 units of Fanta Pineapple bottle"
        },
        {
          "type": "text",
          "text": "Shelf 3:"
        },
        {
          "type": "text",
          "text": "\t- Must have 8 units of Coca-Cola original can 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 4:"
        },
        {
          "type": "text",
          "text": "\t- Must have 8 units of Coca-Cola original can 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 5:"
        },
        {
          "type": "text",
          "text": "\t- Must have 8 units of Coca-Cola original can 330ml"
        },
        {
          "type": "text",
          "text": "Shelf 6 (bottom)"
        },
        {
          "type": "text",
          "text": "\t- Must have 16 units of Dasani Water bottle 550ml"
        },
        {
          "type": "text",
          "text": "\n"
        },
        {
          "type": "text",
          "text": "#Sample product images: \n Fanta Pineapple bottle image: "
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_fanta_pineapple_image}"
          }
        },
        {
          "type": "text",
          "text": "\n Sprite bottle image: "
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_sprite_image}"
          }
        },
        {
          "type": "text",
          "text": "\n Dasani bottle image: "
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_dasani_image}"
          }
        },
        {
          "type": "text",
          "text": "Planogram image: \n"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_shelf_image}"
          }
        },
        {
          "type": "text",
          "text": "\n"
        }
      ]
    }
  ],
  "temperature": 0.5,
  "top_p": 0.95,
  "max_tokens": 800,
}

In [ ]:
ENDPOINT_BASE = os.getenv('AZURE_OPENAI_ENDPOINT')
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
MODEL_DEPLOYMENT = os.getenv('AZURE_OPENAI_MODEL_DEPLOYMENT')
ENDPOINT = f"{ENDPOINT_BASE}openai/deployments/{MODEL_DEPLOYMENT}/chat/completions?api-version={API_VERSION}"


# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

In [ ]:
# Extract and print the 'choices' -> 'message' -> 'content' part of the response JSON
choices_content = response.json().get('choices', [{}])[0].get('message', {}).get('content')
print(choices_content)